# VAE Training - Faces dataset

## imports

Hi,
I was using AWS SageMaker built-in kernels for TensorFlow, and installing the missing pieces via pip install.

Today, I followed the approach recommended by the book, namely:

### CPU version
In my Jupyter NB terminal, did

- `conda create -n hba-gan python=3.6 ipykernel`
- `conda activate hba-gan`
- `pip install -r requirements.txt`
- `conda deactivate`

Then, from the Jupyter NB, I selected the 'hba-gan' kernel which I created above, and everything works fine. So, I'm closing this issue.

### GPU version


- `conda create -n hba-gan-gpu-tf14 python=3.6 ipykernel`
- `conda activate hba-gan-gpu-tf14`
- `pip install -r requirements-PA2-2020-11-07gpu-hba.txt`
- `conda deactivate`

## CelebA Dataset Download Insturctions

Dataset: http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html

Instructions Source: https://github.com/davidADSP/GDL_code/issues/10

CelebA: http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html
You can get the files from the link to Google Drive or Baidu Drive. They are public datasets so no cost to download but you will need a Google account to access Google Drive.

You will also need to download the feature list csv file, list_attr_celeba.csv. I found it here: https://github.com/togheppi/cDCGAN/blob/master/list_attr_celeba.csv. You have to change the first header field from "filename" to "image_id". You can get it

### CelabA

`
Get a google account if you don't have one.
Go to the celebA website
Select the Aligned&Cropped Images. This will take you to the Google Cloud Drive site.
Sign in with your Google account
Download
Colab Notebooks > GAN > CelebA > Anno > list_attr_celeba.txt
Colab Notebooks > GAN > CelebA > Img > img_align_celeba.zip
Unzip img_align_celeba.zip to path/to/GDL_Code/data/celeb this will create img_align_celeba/*jpg
Move the list_attr_celeba.txt to path/to/GDL_Code/data/celeb/list_attr_celeba.csv.
Delete the first line with count of the number of lines in the file (202599).
Prepend the header line that start with, "5_o_Clock_Shadow" with "image_id,...".
That's it for celebA.
The QuickDraw dataset:
`

### With gsutil cp:

`
Install gsutil into your python package. On Windows I'm using Anaconda 3 Python but still used pip:
pip install gsutil
Be careful if you are on Linux and install from a deb or rpm. There is another utility with the same name. Also, if you installed Anaconda on Windows for all users, make sure you run pip in a command windows as Administrator, otherwise the installation will fail.
The gsutil command is
gsutil cp -r gsutil cp -r gs://quickdraw_dataset/full/numpy_bitmap/camel.npy . local_path/to/GDL_Code/data/camel/```

From the browser:

You will need a Google account.

Go to the quickdraw numpy dataset bucket in the GCP browser.

Select camel.np and click on the ... and download the file to your data/camel directory.
`

In [ ]:
%reload_ext autoreload
%autoreload 2

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11101085406684175440
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9807163541248138172
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6630097006971359543
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:2"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16755987584792597078
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:3"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13573526370188989653
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:4"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15953176375870962206
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:5"
device

In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.
W1107 17:38:47.578044 140226616198976 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1107 17:38:47.578913 140226616198976 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1107 17:38:47.579619 140226616198976 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1107 17:38:47.647608 140226616198976 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/kera

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1',
 '/job:localhost/replica:0/task:0/device:GPU:2',
 '/job:localhost/replica:0/task:0/device:GPU:3',
 '/job:localhost/replica:0/task:0/device:GPU:4',
 '/job:localhost/replica:0/task:0/device:GPU:5',
 '/job:localhost/replica:0/task:0/device:GPU:6',
 '/job:localhost/replica:0/task:0/device:GPU:7']

In [3]:
import os
from glob import glob
import numpy as np

from models.VAE import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator


In [4]:
# run params
section = 'vae'
run_id = '0001'
data_name = 'faces'
RUN_FOLDER = 'run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #


DATA_FOLDER = './data/celeba/'

## data

In [5]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)

In [6]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training"
                                            )

Found 202599 images belonging to 1 classes.


## architecture

In [7]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

W1107 17:39:08.330542 140226616198976 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1107 17:39:08.361097 140226616198976 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W1107 17:39:08.439363 140226616198976 deprecation.py:506] From /home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
vae.encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 

In [8]:
vae.decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              823296    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0         
__________

## training

In [9]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10000
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [10]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

W1107 17:39:15.477465 140226616198976 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [ ]:
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
   4/6331 [..............................] - ETA: 13:39:33 - loss: 1257.6749 - vae_r_loss: 1054.8738 - vae_kl_loss: 202.8011

/home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.175932). Check your callbacks.
  % delta_t_median)


 402/6331 [>.............................] - ETA: 39:59 - loss: 508.5842 - vae_r_loss: 420.3447 - vae_kl_loss: 88.2395

/home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.101491). Check your callbacks.
  % delta_t_median)


 405/6331 [>.............................] - ETA: 39:55 - loss: 507.6808 - vae_r_loss: 419.5292 - vae_kl_loss: 88.1516

...

2059/6331 [========>.....................] - ETA: 21:44 - loss: 220.1341 - vae_r_loss: 161.1773 - vae_kl_loss: 58.9568

/home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.100170). Check your callbacks.
  % delta_t_median)


2080/6331 [========>.....................] - ETA: 21:38 - loss: 220.1542 - vae_r_loss: 161.1904 - vae_kl_loss: 58.9638

/home/ec2-user/anaconda3/envs/hba-gan-gpu-tf14/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.104436). Check your callbacks.
  % delta_t_median)


2717/6331 [===========>..................] - ETA: 18:24 - loss: 220.0508 - vae_r_loss: 161.1101 - vae_kl_loss: 58.9407